<a href="https://colab.research.google.com/github/liron7722/AI-Generated-Text-Detector/blob/Production/tdIdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Load CSV
df = pd.read_csv('https://uc730d9484c996374ded5251c19a.dl.dropboxusercontent.com/cd/0/get/Cf-4t8hV3cNA48c-apaehUhFlzd6AF2Et3qaxMown3Kdd3TndMCXU5fuIrZcnVbWIweK-nR7M2AZUgT48yMpcVALQhFyth7IXBQFcYtu_zwhq4r5qek1R1thC-IXL5yBxQcMCClEkJi2UEsdoDw9sBiAslzEJJ-ZXhfl7QquStpOKQ/file?_download_id=32597468389495576265761807982472667737099252508555987932012252364&_log_download_success=1#')

# Calculate TF-IDF
vectorizer = TfidfVectorizer(max_features=250, ngram_range=(2,4))
tfidf_matrix = vectorizer.fit_transform(df['text'])

# Convert to DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=vectorizer.get_feature_names_out()
)

# Save to CSV
tfidf_df.to_csv('tfidf.csv', index=False)